# Autoscout24 webscraper
## The future of electric cars
### Scope
Electric cars in The Netherlands (with the abilaty to progress into The Benelux)

### Motivation
Buying a car is a quite time-consuming investment, buyers stay around 3 months in the market before purchasing their car (Cox Automotive, 2018). According to research in the United States 95% of the people that bought a car, used a digital source of information to find their car (Schueller, 2017). Autoscout – of German descent – helps people find their car and is the biggest online car market in Europe with more than 2 million cars on their website.

In [1]:
# import required packages
import requests
from bs4 import BeautifulSoup
from time import sleep
from datetime import datetime
import csv
import json
import pandas as pd

In [2]:
# empty lists
cars = []
new_cars = []
urls = []
jsondic = []

In [3]:
# looping through the pages
for page in range(1,21):
    for year in range(2011,2023):
        for bcol in range(1,17):
            url_to_get = "https://www.autoscout24.nl/lst/?sort=standard&desc=0&bcol="+str(bcol)+"&fuel=E&ustate=N%2CU&size=20&page="+str(page)+"&cy=NL&fregto="+str(year)+"&fregfrom="+str(year)+"&atype=C&fc=7&qry="
            urls.append(url_to_get)

In [5]:
# retrieving data from homepage
for url in urls:
    pages = requests.get(url)
    soup = BeautifulSoup(pages.text, 'html.parser')
    autoscout_data = soup.find_all('div', attrs = {'class': "cl-list-element cl-list-element-gap"})
    sleep(3)
    for store in autoscout_data:
        new_car = {}
        
        if store.find(attrs={"data-articleid": True}) == None:
            pass
        else:
            item = store.find(attrs={"data-articleid": True})
            ID = (item['data-articleid'])
            new_car['ID'] = ID
        
        if store.find("h2", {"class": "cldt-summary-makemodel sc-font-bold sc-ellipsis"}) == None: 
            pass
        else:
            naam = store.find("h2", {"class": "cldt-summary-makemodel sc-font-bold sc-ellipsis"}).text.strip(None)
            new_car['naam'] = naam
        
        if store.find("h2", {"class": "cldt-summary-version sc-ellipsis"}) == None: 
            pass
        else:
            sub_name = store.find("h2", {"class": "cldt-summary-version sc-ellipsis"}).text.strip(None)
            new_car['beschrijving'] = sub_name
            
        if store.find("span", {"class": "cldt-price sc-font-xl sc-font-bold"}) == None: 
            pass
        else:
            price = store.find("span", {"class": "cldt-price sc-font-xl sc-font-bold"}).text.strip(None)
            price = price.replace("\nBTW verrekenbaar", "")
            new_car['prijs'] = price  
        
        if store.find("li", {"data-type": "first-registration"}) == None: 
            pass
        else:
            year = store.find("li", {"data-type": "first-registration"}).text.strip(None)
            new_car['jaar_maand'] = year
        
        if store.find("ul", {"data-item-name": "vehicle-details"}).find_all("li")[2] == None: 
            pass
        else:
            power = store.find("ul", {"data-item-name": "vehicle-details"}).find_all("li")[2].text.strip(None)
            new_car['power'] = power
        
        if store.find("li", {"data-type": "previous-owners"}) == None: 
            pass
        else:
            owners = store.find("li", {"data-type": "previous-owners"}).text.strip(None)
            new_car['eigenaren'] = owners
        
        if store.find("li", {"data-type": "offer-type"}) == None: 
            pass
        else:
            condition = store.find("li", {"data-type": "offer-type"}).text.strip(None)
            new_car['conditie'] = condition
        
        if store.find("li", {"data-type": "mileage"}) == None: 
            pass
        else:
            kilometer = store.find("li", {"data-type": "mileage"}).text.strip(None)
            new_car['kilometerstand'] = kilometer
        
        if store.find("li", {"data-type": "transmission-type"}) == None: 
            pass
        else:
            transmission = store.find("li", {"data-type": "transmission-type"}).text.strip(None)
            new_car['transmissie'] = transmission
            
        if store.find("li", {"class": "summary_item_no_bottom_line"}) == None: 
            pass
        if store.find("li", {"class": "summary_item_no_bottom_line"}).find("li", {"data-type": "co2-emission"}):
            pass
        if store.find("li", {"class": "summary_item_no_bottom_line"}).find("li", {"data-type": "combined-consumption"}):
            pass
        else:
            energy = store.find("li", {"class": "summary_item_no_bottom_line"}).text.strip(None)
            new_car['brandstof'] = energy
            
        if store.find(class_="cldf-summary-seller-company-name") == None: 
            pass
        else:
            seller_name = store.find(class_="cldf-summary-seller-company-name").text.strip(None)
            new_car['naam_verkoper'] = seller_name
        
        if store.find("span", {"class": "cldf-summary-seller-contact-zip-city"}) == None: 
            pass
        else:
            seller_adress = store.find("span", {"class": "cldf-summary-seller-contact-zip-city"}).text.strip(None)
            new_car['adres_verkoper'] = seller_adress
        
        if store.find("span", {"class": "cldf-summary-seller-contact-country"}) == None: 
            pass
        else:
            seller_country = store.find("span", {"class": "cldf-summary-seller-contact-country"}).text.strip(None)
            new_car['land_verkoper'] = seller_country
        
        # retrieving the link for further extraction
        if store.find('a', href=True) == None: 
            pass
        else:
            link = store.find('a', href=True)
            if link['href'].startswith('/aanbod'):
                link = "https://www.autoscout24.nl" + link['href']
                new_car['link'] = link
    
        # retrieving scrape date and time for validity
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        current_date = now.strftime("%d/%m/%Y")
        new_car['tijd_van_scrapen'] = current_time
        new_car['datum_van_scrapen'] = current_date
       
        cars.append(new_car)
    
    
sleep(3)

In [7]:
# retrieving information from detailed productpage
for car in  cars:
    req = requests.get(car['link'])
    bs = BeautifulSoup(req.text, "html.parser")
    sleep(3)
    
    information = {}
    if bs.find(class_ = "cldt-categorized-data cldt-data-section sc-pull-right") == None: 
        pass
    else:
        attributes = bs.find(class_ = "cldt-categorized-data cldt-data-section sc-pull-right").find_all('dt')
    
    if bs.find(class_ = "cldt-categorized-data cldt-data-section sc-pull-right") == None: 
        pass
    else:
        values = bs.find(class_ = "cldt-categorized-data cldt-data-section sc-pull-right").find_all('dd')
    
    for x, y in zip(attributes, values):
        information[x.text]=y.text
    for key, value in information.items():
        information[key] = value.strip("\n")
    
    car['informatie'] = information
    
    if bs.find(class_="cldt-equipment-block sc-grid-col-3 sc-grid-col-m-4 sc-grid-col-s-12 sc-pull-left") == None: 
        pass
    else:
        comfort = bs.find(class_="cldt-equipment-block sc-grid-col-3 sc-grid-col-m-4 sc-grid-col-s-12 sc-pull-left").text.strip(None)
        comfort = comfort.replace("\n", ", ")
        comfort = comfort.replace(",", " ")
        comfort = comfort.replace("Comfort en gemak", "")
        car['comfort'] = comfort
        
    if bs.find("div", {"data-type": "description"}) == None: 
        pass
    else:
        text = bs.find("div", {"data-type": "description"}).text.strip(None)
        car['tekst'] = text
    
    new_cars.append(car)


sleep(3)                  
       

In [8]:
# save to csv
keys = new_cars[0].keys()

with open('autoscout24_out_new3.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(new_cars)

In [9]:
# parsing the JSON column
with open('autoscout24_out_new3.csv','r',encoding='utf-8') as file:
    reader = csv.reader(file)
    cnt=0
    for row in reader:
        cnt+=1
        if cnt==1: 
            colnames = row
            continue
        
        jsonobj = json.loads(row[17].replace("'", '"').replace("\\", "\\\\"))
       
        
        for c in range(len(colnames)):
            if (c==17): continue
            jsonobj[colnames[c]]=row[c]
            
        jsondic.append(jsonobj)

In [10]:
df_new = pd.DataFrame.from_dict(jsondic)
df_new.set_index('ID', inplace=True)
# delete licensenumber regarding privacy
del df_new['Advertentienr.']

In [11]:
df_new.to_csv('autoscout24_outfile_def.csv')